# Open Data Sabadell


http://opendata.sabadell.cat/ca/

From main page **Catàleg** we select **Medi Ambient** (Environment) category.  
In there, we can find a file related to municipal waste that can be downloaded from [here.](http://opendata.sabadell.cat/index.php?option=com_iasopendata&view=download&format=raw&urlOData=aHR0cDovL29kYXRhLnNhYmFkZWxsLmNhdC9vZGF0YTRQcm9kdWN0b3Ivb2RhdGE0UHJvZHVjdG9yLnN2Yy9NYXRlcmlhbHNSZXNpZHVzLz9mb3JtYXQ9Y3N2JmlkZGlzdD0xNDYyJiRzZWxlY3Q9T3JkcmUsQW55byxJZE1hdGVyaWFsLE5vbU1hdGVyaWFsLFF1YW50aXRhdCxVbml0YXRz) But you can find a copy of this file named **residus.csv** in this tutorial dataset in **DadesSabadell** folder.

The file obtained is in a CSV format (Comma Separated Value). This means that it is 
like a table where the rows are registers and the columns are fields or values associated to this register. 

You can take a look into this file in table-like representation provided in the open data web page following this link: [OpenData Sabadell](http://opendata.sabadell.cat/ca/inici/odata?iddist=1462). Looking this table we can observe that every register represents an amount of waste origined in Sabadell in tones associated with a particular year and a waste type classification.

In this exercice we are going to follow this steps:
* Read residus.csv file.
* Group data to obtain more readable information. Every row will be a type of waste and the columns will be a year related to this data.
* Represent data in an interactive way.
    
**Important:** Nan values indicate that there is no infomation related to this type of waste and year.


To read data and manage it we are going to use [pandas](https://pandas.pydata.org/) library:

In [ ]:
import pandas as pd

df = pd.read_csv("DadesSabadell/residus.csv",sep=';')
df.sort_values(by="Anyo",inplace = True)

df

In [ ]:
materials = list(df.NomMaterial.unique())
materials.remove("Resta")

print ('{} diferent types of wastes:'.format(len(materials)))
materials

In [ ]:
pd.pivot_table(df,columns="Anyo", index = "NomMaterial", values="Quantitat")

All is prepared to plot our data. We are going to us Dash (web-based interfaces in Python) to plot information and give them interactivity.

If you are interested to learn more about Dash you can follow the offical [tutorial.](https://dash.plot.ly/)

[Dash installation:](https://dash.plot.ly/installation)

* **pip install dash**  # The core dash backend
* **pip install dash-html-components**  # HTML components
* **pip install dash-core-components**  # Supercharged components
* **pip install dash-table**  # Interactive DataTable component (new!)

In [ ]:
%%writefile my_app1.py
#Dash empty structure
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()

app.layout = html.Div('Hello Dash!')

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
!python3 my_app1.py

In [ ]:
%%writefile my_app2.py
#Dash simple example
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()

app.layout = html.Div([html.H1("Hello Dash!"),
                       
              html.Div('''
                Dash: A web application framework for Python.
              '''),
                      
              dcc.Graph(id='exmaple-graph',
                        figure = {
                            'data':[
                                {'x': [1, 2, 3], 'y': [4.2, 1.8, 2.7], 'type': 'bar', 'name': 'Sabadell'},
                                {'x': [1, 2, 3], 'y': [2.8, 4.9, 5.1], 'type': 'bar', 'name': 'Barcelona'},
                            ],
                            'layout':{
                                'title' : 'Dash Data Visualisation'
                            }
              })
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
!python3 my_app2.py

In [ ]:
%%writefile my_app3.py
#Dash simple example
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from plotly import tools

df = pd.read_csv("DadesSabadell/residus.csv",sep=';')
df.sort_values(by="Anyo",inplace = True)

materials = list(df.NomMaterial.unique())
materials.remove("Resta")

pd.pivot_table(df,columns="Anyo", index = "NomMaterial", values="Quantitat")

minim = 0
maxim = 1000
pas = 100

app = dash.Dash()
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

app.layout = html.Div([html.Div([dcc.Graph(id='residus_graph')],
                                style={'height':'80%','padding': '0px 20px 20px 20px'}),
                                html.Div([html.H5("Materials amb mitjana de tones per any més grans que:"),
                                    dcc.Slider(id='avg-tones',step = pas,min=minim,max=maxim,value=maxim/2,       
                                            marks={ str(tones): {'label':str(tones)} for tones in range(minim,maxim+pas,pas)})],
                       style={'margin':'auto','height':'20%','width': '70%', 'padding': '0px 0px 40px 40px',"display":'inline_block'})])

@app.callback(
    dash.dependencies.Output('residus_graph', 'figure'),
    [dash.dependencies.Input('avg-tones', 'value')])

def update_figure(avg_tones):
    fig = tools.make_subplots(rows=2, cols=1,shared_xaxes=True, vertical_spacing=0.001)
    traces = []
    
    filtered= df[df['NomMaterial'] =="Resta"]
    trace_Resta = go.Scatter(
                            x=filtered['Anyo'],y=filtered['Quantitat'],text="Resta",
                            mode='lines+markers',
                            opacity=0.7,
                            marker={
                                'size': 15,
                                'line': {'width': 0.5, 'color': 'white'}
                            },
                            name="Resta"
                        )  
    
    filtered = df[df['NomMaterial'].isin(materials)].groupby("Anyo").sum()
    trace_Total= go.Scatter(                      
                            x=filtered.index,y=filtered['Quantitat'],text="Total Materials",
                            mode='lines+markers',
                            opacity=0.7,
                            marker={
                                'size': 15,
                                'line': {'width': 0.5, 'color': 'white'}
                            },
                            name="Total Materials",
                            
                        )   
    
    fig.append_trace(trace_Resta, 1, 1)
    fig.append_trace(trace_Total, 1, 1)
    
    for i in materials :
        filtered= df[df['NomMaterial'] == i]
        y=filtered['Quantitat']
        if (y.mean()>avg_tones) :
            x=filtered['Anyo']
            traces.append(go.Scatter(x=x,y=y,text=i,mode='markers',opacity=0.7,
                                marker={
                                    'size': 15,
                                    'line': {'width': 0.5, 'color': 'white'}
                                }, name=i[0:30])) 
    for trace in traces:
        fig.append_trace(trace, 2, 1)


    fig['layout'].update(height=600,title="Residus a Sabadell", margin={'l': 50, 'b': 40, 't': 40, 'r': 50},
              yaxis1={"title":"Totals"}, yaxis2={"title":"Materials"},hovermode="closest")
    return fig


if __name__ == '__main__':
    app.run_server()

In [ ]:
!python3 my_app3.py

# Open Data Barcelona 

**EXERCICE**

Using information of 2017's births in Barcelona Districts that you can find at [Open Data Barcelona](http://opendata-ajuntament.barcelona.cat/en/) that you can download [here](http://opendata-ajuntament.barcelona.cat/data/en/dataset/est-demo-naixements-sexe), try to obtain a fancy plot showing girls and boys births per different Barcelona District(Slicer is not necessary). 

You also can find this the CSV file with this information in `DadesBarcelona/2017_naixements_sexe.csv`



In [ ]:
import pandas as pd

df = pd.read_csv("DadesBarcelona/2017_naixements_sexe.csv",sep=',')

df

In [ ]:
#pd.pivot_table()

In [ ]:
%%writefile my_exercice.py
#Dash exercice
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from plotly import tools
import numpy as np

df = pd.read_csv("DadesBarcelona/2017_naixements_sexe.csv",sep=',')
#pv = pd.pivot_table() ADD YOUR CODE

app = dash.Dash()
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

app.layout = html.Div([html.Div(
    [dcc.Graph(id='birth_bcn',figure = {
        
                                    #YOUR CODE HERE
         
                              } )
                                ],
                                style={'height':'80%','padding': '0px 20px 20px 20px'},
                                
                               )])




if __name__ == '__main__':
    app.run_server(debug=True)


if __name__ == '__main__':
    app.run_server()

In [ ]:
!python3 my_exercice.py